In [1]:
# Import libraries:
import pandas as pd
import dt_ids7_export_utils as bh_utils
import elfys_functions as bh_elfys

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'RRH ELfys Ids7.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH Elfys reduced.xlsx')

del path


# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)

Dropping unnecessary column: Prioritet- og lesemerkeikon
Dropping unnecessary column: Lagt til i demonstrasjon-ikon
Dropping unnecessary column: Status
Number of rows with NaT in the column "Bestilt dato og tidspunkt": 0
Number of cancelled procedures: 191
Number of non-human subjects: 0
Number of rows with invalid accession number: 0
Number of accession numbers in IDS7: 2046
Number of accession numbers in IDS7 not in DoseTrack: 241


In [4]:
bh_utils.export_examination_codes_to_text_file(df_ids7, 'KRH_Elfys1')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'KRH_Elfys2')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'KRH_Elfys3')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'KRH_Elfys4')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'KRH_Lab13')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'KRH_Lab39')

In [ ]:
# Create a new dataframe which includes Accession Number from ids7, a mapped procedure name, and dose information:
df_dose = bh_elfys.create_dose_dataframe(df_ids7, df_dt, True)


In [2]:
# First we perform mapping on selected procedures:
df_ids7 = bh_elfys.map_procedures(df_ids7, True)

Mapping procedures...

RGA Cor Ablasjon SVT (int.) -> RGA Cor Ablasjon SVT (int.) m og u 3D
RGA Cor Ablasjon SVT m 3D (int.) -> RGA Cor Ablasjon SVT (int.) m og u 3D
RGA Cor Ablasjon Atrieflimmer (int.) -> RGA Cor Ablasjon Atrieflimmer (int.) m og u 3D
RGA Cor Ablasjon Atrieflimmer med 3D (int.) -> RGA Cor Ablasjon Atrieflimmer (int.) m og u 3D
RGA Cor Elfys SVT (int.) -> RGA Cor Elfys VT el. SVT (int.)
RGA Cor Elfys VT (int.) -> RGA Cor Elfys VT el. SVT (int.)
RGA Cor 2-k PM (int.) -> RGA Cor Implantasjon PM/ICD (int.) ink. 2k og 1k PM
RGA Cor 1-k PM (int.) -> RGA Cor Implantasjon PM/ICD (int.) ink. 2k og 1k PM
RGA Cor Implantasjon PM/ICD (int.) -> RGA Cor Implantasjon PM/ICD (int.) ink. 2k og 1k PM


In [2]:
df_dose = df_ids7.groupby('Henvisnings-ID').agg({'Pasient': 'first', 
                                                 'Henvisnings-ID': 'first', 
                                                 'Beskrivelse': bh_utils.concatenate_protocol})

In [24]:
df_ids7['Beskrivelse'].str.contains('RGA Cor Ablasjon').any()

False

In [ ]:
# Check for patients with multiple bookings on the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_time_with_different_accession(df_ids7)

In [3]:
# Check for patients with multiple bookings on the same day but not the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_day_with_different_accession(df_ids7)